# 1. Config

In [2]:
import os
import tensorflow as tf

In [3]:
# local or colab
drive_mode = True

# images scaled and packed in archive of var name @IMAGES_ARCHIVE
archived_images = True

In [4]:
LABELS = [
    {'name':'bike', 'id':1}, 
    {'name':'scooter', 'id':2},
    {'name':'rolls', 'id':3},
    {'name':'pedestrian', 'id':4},
    {'name':'uto', 'id':5}
]

In [20]:
# folders

# paths to images on instance 
IMAGES = os.path.join('images')
IMAGES_TEST = os.path.join('images', 'test')
IMAGES_TRAIN = os.path.join('images', 'train')

TENSORFLOW_MODELS = os.path.join('external', 'tensorflow_models')
TF_RECORDS = os.path.join('external', 'tf_records')

DRIVE_NAME = os.path.join('drive')
GOOGLE_DRIVE = os.path.join(DRIVE_NAME, 'MyDrive')
GOOGLE_DRIVE_COLAB = os.path.join(GOOGLE_DRIVE, 'COLAB')
GOOGLE_DRIVE_IMAGES = os.path.join(GOOGLE_DRIVE, 'projects','images')

IMAGES_ARCHIVE = os.path.join('archive.tar.gz')
GOOGLE_DRIVE_IMG_ARCHIVE = os.path.join(GOOGLE_DRIVE, 'projects', IMAGES_ARCHIVE)

# files
LABEL_FILE = os.path.join('label_map.txt')
TF_RECORD_FILE = os.path.join(TF_RECORDS, 'generate_tfrecord.py')
TRAIN_RECORD = 'train.record'
TEST_RECORD = 'test.record'
VAL_RECORD = 'val.record'

# 2. Download and prepare required soft

In [6]:
if drive_mode:
  from google.colab import drive
  drive.mount(DRIVE_NAME)

Mounted at drive


In [24]:
# get from github
if not os.path.exists(TENSORFLOW_MODELS):
  !git clone --depth 1 https://github.com/tensorflow/models {TENSORFLOW_MODELS
if not os.path.exists(TF_RECORDS):
  !git clone --depth 1  https://github.com/nicknochnack/GenerateTFRecord {TF_RECORDS}

In [ ]:
# exit code should be equal 0
if os.system(f" \
cd {TENSORFLOW_MODELS}/research; \
protoc object_detection/protos/*.proto --python_out=.; \
cp object_detection/packages/tf2/setup.py .; \
python -m pip install .") != 0:
  raise Exception("Can't install TF object detection API")

In [ ]:
# run model builder test
!python {TENSORFLOW_MODELS}/research/object_detection/builders/model_builder_tf2_test.py

In [ ]:
!nvidia-smi
if tf.test.gpu_device_name() == '':
    raise Exception('GPU IS MISSING')

In [ ]:
tf.keras.backend.clear_session()
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        assert tf.config.experimental.get_memory_growth(gpus[0])
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
        

# 3. Images

In [31]:
# copy images
if drive_mode and archived_images:
  !cd drive/MyDrive/projects/ && ls
  !cp drive/MyDrive/projects/archive.tar.gz ./
elif drive_mode and not archived_images:
  !cp -r {GOOGLE_DRIVE_IMAGES} {IMAGES}

archive.tar.gz


In [ ]:
# unpack images if archived
if drive_mode and archived_images:
  if os.path.exists(IMAGES_ARCHIVE) and not os.path.exists(IMAGES):
    !tar -zxvf {IMAGES_ARCHIVE}

## 4. Labels and records

In [13]:
with open(LABEL_FILE, 'w') as f:
    for label in LABELS:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [ ]:
# tf record file generate
!python {TF_RECORD_FILE} -x {IMAGES}/train -l {LABEL_FILE} -o {TRAIN_RECORD} 
!python {TF_RECORD_FILE} -x {IMAGES}/test -l {LABEL_FILE} -o {TEST_RECORD}
!python {TF_RECORD_FILE} -x {IMAGES}/val -l {LABEL_FILE} -o {VAL_RECORD}